# Course 4: Planning
## Part 1: Behaviour Planning in C++
#### By Jonathan L. Moran (jonathan.moran107@gmail.com)
From the Self-Driving Car Engineer Nanodegree programme offered at Udacity.

## Objectives
* TODO.

## 1. Introduction

### 1.1. Vehicle Motion Planner: The Basics

The planning system of a vehicle, as shown below, describes a set of complex, interdependent modules responsible for e.g., localising the vehicle, planning the trajectory, performing sensor fusion, etc. These modules are often interweaved, as their inputs / outputs are dependent on the other modules in the planning stack.

<img src="4-1-Behavior-Planning/figures/2022-12-14-Figure-1-Taxonomy-of-a-Motion-Planner.png" width="80%" height="80%" alt="Figure 1. The Taxonomy of a Motion Planner: set of modules in the planning system, linked together by inputs / outputs and a timing mechanism. Here the refresh rate of each module increases from top-to-bottom.">

$$
\begin{align}
\textrm{Figure 1. The Taxonomy of a Motion Planner: set of modules in the planning system, linked together by inputs / outputs and a timing mechanism. Here the refresh rate of each module increases from top-to-bottom.}
\end{align}
$$

In the figure above we see that the behaviour planning module (top) has the lowest refresh rate, while the motion control module has the highest. From the diagram we also see that some modules depend on the results (outputs) of another. For example, the behaviour planning module (top) takes as input the prediction and the localisation module outputs, whereas the trajectory planning module takes as inputs the behaviour module and the prediction module outputs.   

### 1.2. Behaviour Planning

##### Vehicle States

In order to plan the behaviour of a vehicle, we must first define the vehicle manoeuvres (states) that are either currently being executed or are intended to be executed on the next time-step. For example, we might enumerate the following states to describe the possible ego-vehicle behaviour in a typical highway driving scenario:
* Follow Vehicle;
* Pass Vehicle;
* Slow down;
* Keep Lane ($\mathrm{KL}$);
* Lane Change Left ($\mathrm{LCL}$);
* Lane Change Right ($\mathrm{LCR}$);
* Prepare Lane Change Left ($\mathrm{PLCL}$);
* Prepare Lane Change Right ($\mathrm{PLCR}$).

While this isn't intended to be an exhaustive list of all possible cases that could occur on a highway drive, we point out that being overly explicit might complicate the overall design of the system. For example, the Pass Vehicle state can be simplified as a combination of two existing states: Lane Change Left ($\mathrm{LCL}$) and Lane Change Right ($\mathrm{LCR}$). On the other hand, we don't want to _oversimplify_ things, since not every action can be reduced this way. Take, for example, a "Lane Change" state. We wouldn't want to reduce either $\mathrm{LCL}$ or $\mathrm{LCR}$ to just a "Lane Change" state, since we likely need to execute specific vehicle commands; in this case, a left lane change would require a left-turn signal to be activated, whereas the right lane change would require the right-turn signal to be activated. This is something that a single "Lane Change" state could not properly take into account.

It is important to remember that the trade-off here is to keep the state space as small as possible, for maintainability reasons, while still having enough logical states to actually represent all the physical states we need to care about.

##### State Transition Functions

When using the [finite-state machine](https://en.wikipedia.org/wiki/Finite-state_machine) model, a state transition function is needed to move the vehicle from one state to the next. For the self-driving vehicle, these inputs might include:
* Predictions;
* Map data;
* Speed limit information;
* Localisation data;
* Current vehicle state.

Note that the above list does *not* contain the previous state, which is not needed by the state transition function in order to move the vehicle into the next state.

#### Behaviour Planning in Code

One way to implement a transition function is to enumerate all possible (i.e., accessible) next states, then deciding from that which one is the most optimal to choose. In order to make this decision, we need to use a [cost function](https://en.wikipedia.org/wiki/Loss_function). With a cost function, we can assign each trajectory a relative score determining how "costly" it may be to execute. From this we can select the trajectory with the minimum associated cost.

In code, a trivial state transition function may look something like:
```python
import sys    # For `sys.maxsize`

def transition_function(
        predictions,
        current_fsm_state,
        current_pose,
        cost_functions,
        weights
):
    """Returns the next-state with the lowest associated cost.
    
    :param predictions: Output from the vehicle prediction module.
    :param current_fsm_state: Current vehicle state in the finite-state machine.
    :param current_pose: Current position and orientation of the vehicle.
    :param cost_functions: List of cost functions to apply to each possible state.
    :param weights: List of weights to multiply the cost by, one per cost function.
    :returns: Next-state whose generated trajectory produced a minimum associated cost. 
    """
    # Keep track of total cost of each state
    costs = []
    # Fetch only the states which can be reached from the current
    # Here the `successor_states` function evaluates the FSM state space 
    possible_successor_states = successor_states(current_fsm_state)
    # Enumerate each possible next-state
    for state in possible_successor_states:
        # Generate a rough idea of what trajectory we would follow
        # if we were to choose this state
        trajectory_for_this_state = generate_trajectory(state,
                                                        current_pose,
                                                        predictions
        )
        # Calculate the cost associated with each trajectory
        cost_for_state = 0
        for i, cost_function in enumerate(cost_functions):
            # Apply each cost function to the generated trajectory
            cost_from_cost_fn = cost_function(trajectory_for_state,
                                              predictions
            )
            # Multiply the resulting cost by the associated function weight
            fn_weight_val = weight[i]
            cost_for_state += fn_weight_val * cost_from_cost_fn
        # Append the resulting trajectory cost evaluated for all cost functions
        costs.append({'state': state, 'cost': cost_for_state})
    # Find the best possible next-state (state with minimum cost)
    best_next_state = None
    min_cost = sys.maxsize
    for i, state in enumerate(possible_successor_states):
        cost_dict = costs[i]
        cost_val = costs[i]['cost']
        if cost_val < min_cost:
            min_cost = cost_val
            # Set the best state to the one with the lowest cost score
            best_next_state = cost_dict['state']
    return best_next_state
```

In practise, the finite-state machine-based approach is no longer industry-standard in today's self-driving vehicles. However, finite-state machines serve as a relatively simple starting point for path planning, and was historically used on some of the earliest autonomous vehicles, including the [2005 DARPA Grand Challenge](https://en.wikipedia.org/wiki/DARPA_Grand_Challenge_(2005)) 1st Place winner — the [Stanford "Stanley"](https://cs.stanford.edu/group/roadrunner/stanley.html) [1], and many other of its competitors. 

To illustrate a finite-state machine used in real-world autonomous vehicles, here's a diagram from the Chen et al., 2004 paper [2] describing Ohio State University's [TerraMax](https://en.wikipedia.org/wiki/TerraMax) — a contestent landing 5th Place in the 2005 DARPA Grand Challenge.


<img src="4-1-Behavior-Planning/figures/2022-12-14-Figure-2-Example-Finite-State-Machine-for-Autonomous-Vehicles-Chen-2004.png" width="" height="" alt="Figure 2. The finite-state machine diagram published in Chen et al., 2004 paper describing the Ohio State University TerraMax autonomous vehicle.">

$$
\begin{align}
\textrm{Figure 2. The finite-state machine diagram published in Chen et al., 2004 paper describing the Ohio State University TerraMax autonomous vehicle.}
\end{align}
$$

## 2. Programming Task

### 2.1. Implement a Cost Function in C++

In previous quizzes, you designed a cost function to choose a lane when trying to reach a goal in the highway driving scenario:
$$
\begin{align}
\mathrm{cost} = 1 - \mathrm{e}^{-\frac{\vert \Delta d \vert}{\Delta s}}.
\end{align}
$$

Here, $\Delta d$ was the lateral distance between the goal lane and the final chosen lane, and $\Delta s$ was the longitudinal distance from the vehicle current position to the goal position. 

In this exercise, we will implement a cost function in C++ that also includes states for planning a lane change right or left (denoted $\mathrm{PLCR}$ and $\mathrm{PLCL}$, respectively). The cost function should incoporate these states. As input to the cost function, we have the following:
* **Intended lane**: the intended lane for the given behaviour. For the $\mathrm{PLCR}$, $\mathrm{PCLC}$, $\mathrm{LCR}$, and $\mathrm{LCL}$, this corresponds to the lane immediately adjacent to the current (one lane over);
* **Final lane**: the immediate resulting lane of the given behaviour. For $\mathrm{LCR}$ and $\mathrm{LCL}$, this would correspond to one lane over;
* **Distance to the goal**: $\Delta s$, the longitudinal distance from the vehicle to the goal;
* **Goal lane**: lane in which the goal marker is situated.

The goal of this exercise is to modify the $\vert \Delta d \vert$ in the equation above so that it satisfies:
* $\vert \Delta d \vert$ decreases when both the intended and final lane are closer to the goal lane;
* Each possible behaviour state $\mathrm{KL}$, $\mathrm{PLCR}$ / $\mathrm{PLCL}$, $\mathrm{LCR}$ / $\mathrm{LCL}$ has a unique cost;
* The values produced by the cost function are in range $[0, 1]$.  

In [1]:
// From J. Moran's `1_distance_cost.cc`

In [2]:
#include <cmath>


/* Computes the distance cost to the goal state including lane changes.
 *
 * Here the cost increases with both the distance of intended lane from the
 * goal and the distance of the final lane form the goal. The cost also 
 * becomes larger as the vehicle gets closer to approaching the goal (i.e.,
 * the cost increases as longitudinal distance to the goal decreases).
 *
 * @param    goal_lane          Lane number containing the goal marker. 
 * @param    intended_lane      Lane number inteded for given manoeuvre.
 * @param    final_lane         Lane number immediately following manoeuvre. 
 * @param    distance_to_goal   Longitudinal distance from vehicle to goal.
 * @returns  cost               Associated distance cost of planned manoeuvre.
 */
double goal_distance_cost(
    int goal_lane, 
    int intended_lane, 
    int final_lane, 
    double distance_to_goal
) {
  // Compute the weighted lateral remaining distance to the goal lane
  double delta_d = 2.0 * goal_lane - intended_lane - final_lane;
  // Taking absolute value to get magnitude of total remaining distance
  double cost = 1.0 - exp(-(std::abs(delta_d) / distance_to_goal));
  return cost;
}

#### Testing the distance cost function

In [3]:
// From J. Moran's `1_tests.cc`

In [4]:
#include <iostream>

/* Tests the goal-distance cost function.
 * The costs are given for input (`intended_lane`, `final_lane`, `goal_lane`).
 * 
 * For example, we might have the following lane configuration:
 *
 * --------------------------------------------------------------
 *  3 |
 * --------|-----------------------------------------------------
 *  2 |    o
 * --------|------------------|----------------------------------
 *  1 |    o                  o
 * -----------------------------------------------------|--------
 *  0 |                                                 x
 * --------------------------------------------------------------
 * 
 * Where we have a `goal_lane = 0`, and three possible manoeuvres to
 * score in `intended_lane = 2` and `intended_lane = 1`. The distance
 * from `intended_lane = 2` to the `goal_lane = 0` can be e.g., 10m, 
 * while the distances from `intended_lane = 1` to the `goal_lane = 0`
 * can be 10m and e.g., 5m. 
 * 
 * Using our cost function, we want to give the two manoeuvres in lane 1 
 * a lower cost score since they are closer laterally to the goal lane.
 * However, within lane 1, we want to give the manoeuvre farther away from
 * the goal marker a lesser cost score (i.e., cost decreases as longitudinal
 * distance to goal increases). This is to factor in reaction time.
 * 
 * Therefore, the manoeuvre the highest cost will be for (2, 2, 10), i.e.,
 * position in lane 2 with no intended or final lane change closer to goal.
 * Assuming an intention to move into lane 1, i.e., (2, 1, 10), this will
 * be our second highest cost. The next highest will be for (1, 1, 5),
 * which represents position in lane 1 with shorter distance to goal and no
 * intention of moving into goal lane. After that, we have (1, 0, 5), which
 * is the position in lane 1 with intention of moving into goal lane but with
 * shorter distance. This results in a lowest cost score assigned to
 * (1, 0, 10), which represents the position in lane 1 with larger distance to
 * goal and intention of moving into goal lane. This is the most optimal case.
 */
void test_goal_distance_cost() {
  // Integer number of the goal lane (see docstring for example)
  int goal_lane = 0;
  double cost;
  std::cout << "Costs for (intended_lane, final_lane, goal_distance):" << "\n";
  std::cout << "---------------------------------------------------------" << "\n";
  cost = goal_distance_cost(goal_lane, 2, 2, 1.0);
  std::cout << "The cost is " << cost << " for " << "(2, 2, 1.0)" << "\n";
  cost = goal_distance_cost(goal_lane, 2, 2, 10.0);
  std::cout << "The cost is " << cost << " for " << "(2, 2, 10.0)" << "\n";
  cost = goal_distance_cost(goal_lane, 2, 2, 100.0);
  std::cout << "The cost is " << cost << " for " << "(2, 2, 100.0)" << "\n";
  cost = goal_distance_cost(goal_lane, 1, 2, 100.0);
  std::cout << "The cost is " << cost << " for " << "(1, 2, 100.0)" << "\n";
  cost = goal_distance_cost(goal_lane, 1, 1, 100.0);
  std::cout << "The cost is " << cost << " for " << "(1, 1, 100.0)" << "\n";
  cost = goal_distance_cost(goal_lane, 0, 1, 100.0);
  std::cout << "The cost is " << cost << " for " << "(0, 1, 100.0)" << "\n";
  cost = goal_distance_cost(goal_lane, 0, 0, 100.0);
  std::cout << "The cost is " << cost << " for " << "(0, 0, 100.0)" << "\n";
    
}

In [5]:
test_goal_distance_cost()

Costs for (intended_lane, final_lane, goal_distance):
---------------------------------------------------------
The cost is 0.981684 for (2, 2, 1.0)
The cost is 0.32968 for (2, 2, 10.0)
The cost is 0.0392106 for (2, 2, 100.0)
The cost is 0.0295545 for (1, 2, 100.0)
The cost is 0.0198013 for (1, 1, 100.0)
The cost is 0.00995017 for (0, 1, 100.0)
The cost is 0 for (0, 0, 100.0)


### 2.2. Implement an Inefficiency Cost Function in C++ 

In most situations, a single cost function will not be sufficient to produce complex vehicle behaviour. In this exercise, we will implement an additional cost function to handle inefficient target trajectories.

Here the "inefficiency cost" is designed to encourage the ego-vehicle to drive in the fastest possible lane. As input to the cost function, we have the following:
* **Target speed**: desired speed to drive the ego-vehicle at (here speed is unitless);
* **Intended lane**: the intended lane for the given behaviour. For the $\mathrm{PLCR}$, $\mathrm{PCLC}$, $\mathrm{LCR}$, and $\mathrm{LCL}$, this corresponds to the lane immediately adjacent to the current (one lane over);
* **Final lane**: the immediate resulting lane of the given behaviour. For $\mathrm{LCR}$ and $\mathrm{LCL}$, this would correspond to one lane over;
* **Lane speeds**: a vector of estimated speeds, one for each lane number (i.e., `{6, 7, 8, 9}`).

The goal of this exercise is to create a cost function that satisfies the following:
* Cost decreases as both the intended and the final lane are at higher speeds (i.e., move to 'faster' lane, goal achieved);
* Each possible behaviour state $\mathrm{KL}$, $\mathrm{PLCR}$ / $\mathrm{PLCL}$, $\mathrm{LCR}$ / $\mathrm{LCL}$ has a unique cost;
* The values produced by the cost function are in range $[0, 1]$.

In [6]:
// From J. Moran's `2_inefficiency_cost.cc`

In [7]:
/* Computes the inefficiency cost of the goal speed given lane changes. 
 *
 * Here the inefficiency cost increases when trajectories with either 
 * `intended_lane` or `final_lane` have traffic moving slower than the
 * `target_speed`. The speed of each lane is given as a unitless number
 * indexed from the `lane_speeds` vector for each of the lane indices.
 * 
 * @param    target_speed   Desired speed (dimensionless) after manoeuvre. 
 * @param    intended_lane  Index of lane intended for this manoeuvre.
 * @param    final_lane     Index of lane immediately following manoeuvre.
 * @param    lane_speeds    Set of current speeds for each lane by index.
 * @returns  cost           Computed inefficiency cost for the trajectory.
 */
double inefficiency_cost(
    int target_speed,
    int intended_lane,
    int final_lane,
    std::vector<int>& lane_speeds
) {
  // Compute the weighted change in speed from manoeuvres to target
  double delta_s = 2.0 * target_speed;
  delta_s = delta_s - lane_speeds[intended_lane] - lane_speeds[final_lane];
  // Return the ratio of weighted change in speed to target speed
  // i.e., `delta_s` is inversely proportional to the target speed.
  double cost = delta_s / target_speed;
  return cost;
}

#### Testing the inefficiency cost function

In [8]:
// From J. Moran's `1_tests.cc`

In [9]:
/* Tests the inefficiency cost function.
 * Here the expectation is that the greater the change in speed
 * from the target speed will result in a higher inefficiency cost.
 * The higher the cost, the less desirable the trajectory. 
 */
void test_inefficiency_cost() {
  // Target speed of our vehicle
  int target_speed = 10;
  // Lane speeds for each lane
  std::vector<int> lane_speeds = {6, 7, 8, 9};
  // Test cases used for grading - do not change.
  double cost;
  std::cout << "Costs for (intended_lane, final_lane):" << "\n";
  std::cout << "---------------------------------------------------------" << "\n";
  cost = inefficiency_cost(target_speed, 3, 3, lane_speeds);
  std::cout << "The cost is " << cost << " for " << "(3, 3)" << "\n";
  cost = inefficiency_cost(target_speed, 2, 3, lane_speeds);
  std::cout << "The cost is " << cost << " for " << "(2, 3)" << "\n";
  cost = inefficiency_cost(target_speed, 2, 2, lane_speeds);
  std::cout << "The cost is " << cost << " for " << "(2, 2)" << "\n";
  cost = inefficiency_cost(target_speed, 1, 2, lane_speeds);
  std::cout << "The cost is " << cost << " for " << "(1, 2)" << "\n";
  cost = inefficiency_cost(target_speed, 1, 1, lane_speeds);
  std::cout << "The cost is " << cost << " for " << "(1, 1)" << "\n";
  cost = inefficiency_cost(target_speed, 0, 1, lane_speeds);
  std::cout << "The cost is " << cost << " for " << "(0, 1)" << "\n";
  cost = inefficiency_cost(target_speed, 0, 0, lane_speeds);
  std::cout << "The cost is " << cost << " for " << "(0, 0)" << "\n";
}

In [10]:
test_inefficiency_cost()

Costs for (intended_lane, final_lane):
---------------------------------------------------------
The cost is 0.2 for (3, 3)
The cost is 0.3 for (2, 3)
The cost is 0.4 for (2, 2)
The cost is 0.5 for (1, 2)
The cost is 0.6 for (1, 1)
The cost is 0.7 for (0, 1)
The cost is 0.8 for (0, 0)


### 2.3. Implement a Behaviour Planner in C++

In this exercise we implement a behaviour planner and related cost functions for the highway driving scenario. The behaviour planner will use prediction data to set the state of the ego-vehicle to one of five values, then generate a corresponding vehicle trajectory from the following set of possible states:
* **Keep Lane** ($\mathrm{KL}$);
* **Lane Change Left** ($\mathrm{LCL}$);
* **Lane Change Right** ($\mathrm{LCR}$);
* **Prepare Lane Change Left** ($\mathrm{PLCL}$);
* **Prepare Lane Change Right** ($\mathrm{PLCR}$).

The objective of this exercise is to navigate the ego-vehicle thorugh traffic to the goal state in as little time as possible. Since the goal marker is in the slowest moving lane, we choose cost functions and respective weight values that prioritise manoeuvres that include faster moving lanes, whenever possible. The `goal_lane` and speed value `s`, as well as the `traffic_speeds` vector are set in the `test_behaviour_planner.cc` file. We are also given two cost functions, which are implemented in `3_costs.cc`. To complete this exercise, we must:
1. **Implement the `choose_next_state` method** in the `3_vehicle.cc` class;
    * Note the Python implementation from the Sect. 1.2. "Behaviour Planner — you should instead return the best trajectory of the best state, rather than the state itself;
2. **Select appropriate weight values for the provided cost functions in `3_costs.cc`**;
    * Use the vehicle state values from the `get_helper_data()` in your implementation;
3. **Evaluate the planner results**.

This exercise has several functions already implemented. These include:
* `successor_states()`: Returns a vector of possible successor states given the current state evaluated with the Finite-State Machine;
* `generate_trajectories()`: Returns a vector of `Vehicle` objects representing each possible trajectory for the ego-vehicle — if no possible trajectories exist for the given state, this vector is of size `0`;
* `calculate_cost()`: Computes the cost for a given trajectory.
* `get_helper_data()`: Updates the lane indices given the proposed trajectory / manoeuvre.

#### Testing the Behaviour Planner in C++

In [13]:
// From J. Moran's `1_tests.cc`

In [14]:
#include "3_road.h"
#include "3_vehicle.h"
#include <iostream>
#include <vector>


void test_behaviour_planner() {
  // impacts default behavior for most states
  int SPEED_LIMIT = 10;
  // all traffic in lane (besides ego) follow these speeds
  std::vector<int> LANE_SPEEDS = {6, 7, 8, 9}; 
  // Number of available "cells" which should have traffic
  double TRAFFIC_DENSITY = 0.15;
  // At each timestep, ego can set acceleration to value between 
  //   -MAX_ACCEL and MAX_ACCEL
  int MAX_ACCEL = 2;
  // s value and lane number of goal.
  std::vector<int> GOAL = {300, 0};
  // These affect the visualization
  int FRAMES_PER_SECOND = 4;
  int AMOUNT_OF_ROAD_VISIBLE = 40;
  Road road = Road(SPEED_LIMIT, 
                   TRAFFIC_DENSITY, 
                   LANE_SPEEDS
  );
  road.update_width = AMOUNT_OF_ROAD_VISIBLE;
  road.populate_traffic();
  int goal_s = GOAL[0];
  int goal_lane = GOAL[1];
  // Configuration data: 
  // {speed limit, num_lanes, goal_s, goal_lane, max_acceleration}
  int num_lanes = LANE_SPEEDS.size();
  std::vector<int> ego_config = {
      SPEED_LIMIT,
      num_lanes,
      goal_s,
      goal_lane,
      MAX_ACCEL
  };
  road.add_ego(2,0, ego_config);
  int timestep = 0;
  while (road.get_ego().s <= GOAL[0]) {
    ++timestep;
    if (timestep > 100) {
      break;
    }
    road.advance();
    road.display(timestep);
    //time.sleep(float(1.0) / FRAMES_PER_SECOND);
  }
  Vehicle ego = road.get_ego();
  if (ego.lane == GOAL[1]) {
    std::cout << "You got to the goal in " << timestep << " seconds!" << "\n";
    if(timestep > 35) {
      std::cout << "But it took too long to reach the goal. Go faster!" << "\n";
    }
  } else {
    std::cout << "You missed the goal. You are in lane " << ego.lane;
    std::cout << " instead of " << GOAL[1] << "." << "\n";
  }
}

In [ ]:
test_behaviour_planner()

Here we omit the output of this test (it's many lines long!)

## Credits

This assignment was prepared by Benjamin Ulmer and Tobias Roth of Mercedes-Benz Research and Development, 2021 (link [here](https://www.udacity.com/course/self-driving-car-engineer-nanodegree--nd0013)).


References
* [1] Chen, Q. et al. Ohio State University at the 2004 DARPA Grand Challenge: developing a completely autonomous vehicle. IEEE Intelligent Systems. 19(5):8-11. 2004. [doi:10.1109/MIS.2004.48](https://doi.org/10.1109/MIS.2004.48).
* [2] Thrun, S. et al. Stanley: The Robot That Won the DARPA Grand Challenge. In: Buehler, M., Iagnemma, K., Singh, S. (eds) The 2005 DARPA Grand Challenge. Springer Tracts in Advanced Robotics. 36(1):1-43. 2005. [doi:10.1007/978-3-540-73429-1_1](https://doi.org/10.1007/978-3-540-73429-1_1).